In [ ]:
import pandas
import pylab
import numpy
from pandas import *
from pylab import *

import sklearn
import random
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from __future__ import division
import math
import matplotlib.cm as cm
import matplotlib.pyplot as pyplot